In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from collections import Counter
from sklearn.utils import resample
from sklearn import preprocessing

In [2]:
test = pd.read_csv("./additional_files/test.dat", sep= ' ')
movieActors = pd.read_csv("additional_files/movie_actors.dat", sep= '\t',encoding='latin-1')
movieDirectors= pd.read_csv("additional_files/movie_directors.dat", sep= '\t',encoding='latin-1')
movieGenres= pd.read_csv("additional_files/movie_genres.dat", sep= '\t',encoding='latin-1')
movieTags= pd.read_csv("additional_files/movie_tags.dat", sep= '\t',encoding='latin-1')
tags= pd.read_csv("additional_files/tags.dat", sep= '\t',encoding='latin-1')
trnTst= pd.read_csv("additional_files/test.dat", sep= ' ',encoding='latin-1')
train= pd.read_csv("additional_files/train.dat", sep= ' ',encoding='latin-1')
userTaggedMovies= pd.read_csv("additional_files/user_taggedmovies.dat", sep= ' ',encoding='latin-1')

In [ ]:
movieActors

In [ ]:
movieDirectors = movieDirectors.drop(columns=['directorName'])
movieDirectors

In [ ]:
movieActors

In [ ]:
movieActors1 = movieActors.drop(columns=['actorName', 'ranking'])
movieActors1

In [ ]:
tags

In [ ]:
userTaggedMovies

In [3]:
# merge usertag and main train data (userid, movieid)
fintrn = train.merge(userTaggedMovies, on=['userID','movieID'], how ='left')
fintrn

,userID,movieID,rating,tagID
0,75,3,1.0,NaN
1,75,32,4.5,NaN
2,75,110,4.0,NaN
3,75,163,4.0,NaN
4,75,165,4.5,NaN
...,...,...,...,...
653911,71534,42900,4.0,NaN
653912,71534,44555,4.0,NaN
653913,71534,46578,4.0,NaN
653914,71534,61075,5.0,NaN


In [4]:
# merge new train and movietags 
fintrn = fintrn.merge(movieTags, on=['movieID','tagID'], how = 'left')
fintrn

,userID,movieID,rating,tagID,tagWeight
0,75,3,1.0,NaN,NaN
1,75,32,4.5,NaN,NaN
2,75,110,4.0,NaN,NaN
3,75,163,4.0,NaN,NaN
4,75,165,4.5,NaN,NaN
...,...,...,...,...,...
653911,71534,42900,4.0,NaN,NaN
653912,71534,44555,4.0,NaN,NaN
653913,71534,46578,4.0,NaN,NaN
653914,71534,61075,5.0,NaN,NaN


In [5]:
# add genre to new train obtained above
fintrn = fintrn.merge(movieGenres, on='movieID', how = 'left')
fintrn

,userID,movieID,rating,tagID,tagWeight,genre
0,75,3,1.0,NaN,NaN,Comedy
1,75,3,1.0,NaN,NaN,Romance
2,75,32,4.5,NaN,NaN,Sci-Fi
3,75,32,4.5,NaN,NaN,Thriller
4,75,110,4.0,NaN,NaN,Action
...,...,...,...,...,...,...
1712235,71534,46578,4.0,NaN,NaN,Drama
1712236,71534,61075,5.0,NaN,NaN,Drama
1712237,71534,61075,5.0,NaN,NaN,Romance
1712238,71534,62049,4.5,NaN,NaN,Drama


In [6]:
# add director to new train obtained above
fintrn = fintrn.merge(movieDirectors, on='movieID', how = 'left')
fintrn

,userID,movieID,rating,tagID,tagWeight,genre,directorID,directorName
0,75,3,1.0,NaN,NaN,Comedy,donald_petrie,Donald Petrie
1,75,3,1.0,NaN,NaN,Romance,donald_petrie,Donald Petrie
2,75,32,4.5,NaN,NaN,Sci-Fi,siddharth_randeria,Siddharth Randeria
3,75,32,4.5,NaN,NaN,Thriller,siddharth_randeria,Siddharth Randeria
4,75,110,4.0,NaN,NaN,Action,mel_gibson,Mel Gibson
...,...,...,...,...,...,...,...,...
1712235,71534,46578,4.0,NaN,NaN,Drama,1167865-jonathan_dayton,Jonathan Dayton
1712236,71534,61075,5.0,NaN,NaN,Drama,isabel_coixet,Isabel Coixet
1712237,71534,61075,5.0,NaN,NaN,Romance,isabel_coixet,Isabel Coixet
1712238,71534,62049,4.5,NaN,NaN,Drama,michael_radford,Michael Radford


In [ ]:
fintrn = fintrn.merge(movieActors1, on='movieID', how='left')
fintrn

In [7]:
# minmax scaling of userid and movieid (because very huge numbers)
df = fintrn[['userID','movieID']]
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
df.columns=['userID','movieID']
fintrn[['userID','movieID']] = df[['userID','movieID']]


In [8]:
fintrn["movieID"].value_counts()

0.066097    6522
0.005450    4824
0.000000    4765
0.083584    4755
0.045845    4494
            ... 
0.994120       1
0.055902       1
0.962062       1
0.077781       1
0.087914       1
Name: movieID, Length: 9936, dtype: int64

In [9]:
# remove tagid because not imp
fintrn = fintrn.drop(columns=['tagID'])

In [10]:
fintrn

,userID,movieID,rating,tagWeight,genre,directorID,directorName
0,0.0,0.000031,1.0,NaN,Comedy,donald_petrie,Donald Petrie
1,0.0,0.000031,1.0,NaN,Romance,donald_petrie,Donald Petrie
2,0.0,0.000476,4.5,NaN,Sci-Fi,siddharth_randeria,Siddharth Randeria
3,0.0,0.000476,4.5,NaN,Thriller,siddharth_randeria,Siddharth Randeria
4,0.0,0.001674,4.0,NaN,Action,mel_gibson,Mel Gibson
...,...,...,...,...,...,...,...
1712235,1.0,0.715117,4.0,NaN,Drama,1167865-jonathan_dayton,Jonathan Dayton
1712236,1.0,0.937696,5.0,NaN,Drama,isabel_coixet,Isabel Coixet
1712237,1.0,0.937696,5.0,NaN,Romance,isabel_coixet,Isabel Coixet
1712238,1.0,0.952650,4.5,NaN,Drama,michael_radford,Michael Radford


In [11]:
# same merges as done for train dataset
finaltest = test.merge(userTaggedMovies, on=['userID','movieID'], how ='left')
finaltest

,userID,movieID,tagID
0,75,653,NaN
1,75,1527,NaN
2,75,3258,NaN
3,75,7007,NaN
4,78,82,NaN
...,...,...,...
72643,71534,2208,NaN
72644,71534,2997,NaN
72645,71534,4306,NaN
72646,71534,7132,NaN


In [12]:
finaltest

,userID,movieID,tagID
0,75,653,NaN
1,75,1527,NaN
2,75,3258,NaN
3,75,7007,NaN
4,78,82,NaN
...,...,...,...
72643,71534,2208,NaN
72644,71534,2997,NaN
72645,71534,4306,NaN
72646,71534,7132,NaN


In [13]:
finaltest = finaltest.merge(movieTags, on=['movieID','tagID'], how = 'left')
finaltest

,userID,movieID,tagID,tagWeight
0,75,653,NaN,NaN
1,75,1527,NaN,NaN
2,75,3258,NaN,NaN
3,75,7007,NaN,NaN
4,78,82,NaN,NaN
...,...,...,...,...
72643,71534,2208,NaN,NaN
72644,71534,2997,NaN,NaN
72645,71534,4306,NaN,NaN
72646,71534,7132,NaN,NaN


In [14]:
finaltest = finaltest.merge(movieGenres, on='movieID', how = 'left')
finaltest

,userID,movieID,tagID,tagWeight,genre
0,75,653,NaN,NaN,Action
1,75,653,NaN,NaN,Adventure
2,75,653,NaN,NaN,Fantasy
3,75,1527,NaN,NaN,Action
4,75,1527,NaN,NaN,Adventure
...,...,...,...,...,...
189917,71534,7132,NaN,NaN,Comedy
189918,71534,7132,NaN,NaN,Musical
189919,71534,7132,NaN,NaN,Romance
189920,71534,35836,NaN,NaN,Comedy


In [15]:
finaltest = finaltest.merge(movieDirectors, on='movieID', how = 'left')
finaltest

,userID,movieID,tagID,tagWeight,genre,directorID,directorName
0,75,653,NaN,NaN,Action,rob_cohen,Rob Cohen
1,75,653,NaN,NaN,Adventure,rob_cohen,Rob Cohen
2,75,653,NaN,NaN,Fantasy,rob_cohen,Rob Cohen
3,75,1527,NaN,NaN,Action,luc_besson,Luc Besson
4,75,1527,NaN,NaN,Adventure,luc_besson,Luc Besson
...,...,...,...,...,...,...,...
189917,71534,7132,NaN,NaN,Comedy,sam_wood,Sam Wood
189918,71534,7132,NaN,NaN,Musical,sam_wood,Sam Wood
189919,71534,7132,NaN,NaN,Romance,sam_wood,Sam Wood
189920,71534,35836,NaN,NaN,Comedy,judd_apatow,Judd Apatow


In [ ]:
finaltest = finaltest.merge(movieActors1, on='movieID', how='left')
finaltest

In [16]:
finaltest = finaltest.drop(columns=['tagID', 'directorName'])

In [17]:
finaltest

,userID,movieID,tagWeight,genre,directorID
0,75,653,NaN,Action,rob_cohen
1,75,653,NaN,Adventure,rob_cohen
2,75,653,NaN,Fantasy,rob_cohen
3,75,1527,NaN,Action,luc_besson
4,75,1527,NaN,Adventure,luc_besson
...,...,...,...,...,...
189917,71534,7132,NaN,Comedy,sam_wood
189918,71534,7132,NaN,Musical,sam_wood
189919,71534,7132,NaN,Romance,sam_wood
189920,71534,35836,NaN,Comedy,judd_apatow


In [18]:
df = finaltest[['userID','movieID']]
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
df.columns=['userID','movieID']
finaltest[['userID','movieID']] = df[['userID','movieID']]

In [19]:
finaltest

,userID,movieID,tagWeight,genre,directorID
0,0.0,0.010017,NaN,Action,rob_cohen
1,0.0,0.010017,NaN,Adventure,rob_cohen
2,0.0,0.010017,NaN,Fantasy,rob_cohen
3,0.0,0.023444,NaN,Action,luc_besson
4,0.0,0.023444,NaN,Adventure,luc_besson
...,...,...,...,...,...
189917,1.0,0.109556,NaN,Comedy,sam_wood
189918,1.0,0.109556,NaN,Musical,sam_wood
189919,1.0,0.109556,NaN,Romance,sam_wood
189920,1.0,0.550545,NaN,Comedy,judd_apatow


In [20]:
fintrn

,userID,movieID,rating,tagWeight,genre,directorID,directorName
0,0.0,0.000031,1.0,NaN,Comedy,donald_petrie,Donald Petrie
1,0.0,0.000031,1.0,NaN,Romance,donald_petrie,Donald Petrie
2,0.0,0.000476,4.5,NaN,Sci-Fi,siddharth_randeria,Siddharth Randeria
3,0.0,0.000476,4.5,NaN,Thriller,siddharth_randeria,Siddharth Randeria
4,0.0,0.001674,4.0,NaN,Action,mel_gibson,Mel Gibson
...,...,...,...,...,...,...,...
1712235,1.0,0.715117,4.0,NaN,Drama,1167865-jonathan_dayton,Jonathan Dayton
1712236,1.0,0.937696,5.0,NaN,Drama,isabel_coixet,Isabel Coixet
1712237,1.0,0.937696,5.0,NaN,Romance,isabel_coixet,Isabel Coixet
1712238,1.0,0.952650,4.5,NaN,Drama,michael_radford,Michael Radford


In [ ]:
# from sklearn.preprocessing import LabelEncoder
# encoder = LabelEncoder()
# fintrn['directorID'] = encoder.fit_transform(fintrn["directorID"])


In [21]:
# convert to categorical data -> genre and director_id
fintrn.genre = pd.Categorical(fintrn.genre)
fintrn['genre'] = fintrn.genre.cat.codes
fintrn.directorID = pd.Categorical(fintrn.directorID)
fintrn['directorID'] = fintrn.directorID.cat.codes

finaltest.genre = pd.Categorical(finaltest.genre)
finaltest['genre'] = finaltest.genre.cat.codes
finaltest.directorID = pd.Categorical(finaltest.directorID)
finaltest['directorID'] = finaltest.directorID.cat.codes

In [25]:
fintrn.actorID = pd.Categorical(fintrn.actorID)
fintrn['actorID'] = fintrn.actorID.cat.codes

finaltest.actorID = pd.Categorical(finaltest.actorID)
finaltest['actorID'] = finaltest.actorID.cat.codes

In [23]:
# fill nan values in tagweight with 0 to remove error
fintrn['tagWeight'] = fintrn['tagWeight'].fillna(0)


In [24]:
finaltest
# same as above cell
finaltest['tagWeight'] = finaltest['tagWeight'].fillna(0)


In [26]:
# normalize directorID and genre because again huge numbers due to categorical conversion
df = finaltest[['directorID','genre']]
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
df.columns=['directorID','genre']
finaltest[['directorID','genre']] = df[['directorID','genre']]

df = fintrn[['directorID','genre']]
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
df.columns=['directorID','genre']
fintrn[['directorID','genre']] = df[['directorID','genre']]

In [ ]:
# normalize directorID and genre because again huge numbers due to categorical conversion
df = finaltest[['actorID']]
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
df.columns=['actorID']
finaltest[['actorID']] = df[['actorID']]

df = fintrn[['actorID']]
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
df.columns=['actorID']
fintrn[['actorID']] = df[['actorID']]

In [27]:
# chose random n samples to train and assign to df (since dataset is pretty huge)
df = fintrn.sample(n=100000)

In [28]:
# y_train = rating column in df
y_train = df.values[:,2]
y_train

array([3.5, 4.0, 4.5, ..., 2.5, 4.0, 3.5], dtype=object)

In [30]:
df = df.drop(columns=['directorName'])

In [29]:
# remove rating from df
df = df.drop(columns=['rating'])
X = df
X

,userID,movieID,tagWeight,genre,directorID,directorName
238632,0.139241,0.061813,0.0,0.263158,0.352422,Ethan Coen
1229435,0.704012,0.041040,0.0,0.421053,0.712664,Mike Figgis
293200,0.170839,0.113001,0.0,0.368421,0.708123,Michel Gondry
865405,0.477505,0.084014,0.0,0.368421,0.793391,Rainer Werner Fassbinder
83646,0.052869,0.128339,0.0,0.105263,0.592079,Kelly Asbury
...,...,...,...,...,...,...
268763,0.158818,0.054581,0.0,0.368421,0.888496,Sofia Coppola
810154,0.450860,0.106568,0.0,0.736842,0.804490,Richard Curtis
1062131,0.588855,0.042867,0.0,0.526316,0.132442,Anthony Waller
1509634,0.881415,0.002149,0.0,0.210526,0.714934,Mike Nichols


In [ ]:
# df = finaltest[['userID','movieID']]
# x = df.values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# df = pd.DataFrame(x_scaled)
# df.columns=['userID','movieID']
# finaltest[['userID','movieID']] = df[['userID','movieID']]

In [33]:
X = df
X

,userID,movieID,tagWeight,genre,directorID
238632,0.139241,0.061813,0.0,0.263158,0.352422
1229435,0.704012,0.041040,0.0,0.421053,0.712664
293200,0.170839,0.113001,0.0,0.368421,0.708123
865405,0.477505,0.084014,0.0,0.368421,0.793391
83646,0.052869,0.128339,0.0,0.105263,0.592079
...,...,...,...,...,...
268763,0.158818,0.054581,0.0,0.368421,0.888496
810154,0.450860,0.106568,0.0,0.736842,0.804490
1062131,0.588855,0.042867,0.0,0.526316,0.132442
1509634,0.881415,0.002149,0.0,0.210526,0.714934


In [34]:
# train test split
X_train = X.values[:,:]
# y_train = df.values[:,2]
X_test =  finaltest.values[:,:]
y_train

array([3.5, 4.0, 4.5, ..., 2.5, 4.0, 3.5], dtype=object)

In [35]:
X_train.shape

(100000, 5)

In [36]:
X_test.shape

(189922, 5)

In [37]:
type(fintrn)

pandas.core.frame.DataFrame

In [ ]:
fintrn.to_csv('train_dataset_processed.csv', index=False, header=True)

In [ ]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor()
regressor.fit(X_train, y_train)

In [38]:
from sklearn.neighbors import KNeighborsRegressor
regressor = KNeighborsRegressor(n_neighbors=140)
regressor.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=140)

In [39]:
y_pred = regressor.predict(X_test)

In [40]:
len(y_pred)

189922

In [41]:
y_pred

array([3.3785714285714286, 3.3785714285714286, 3.7357142857142858, ...,
       3.407142857142857, 3.4035714285714285, 3.4785714285714286],
      dtype=object)

In [42]:
finaltest["rating"] = list(y_pred)

In [43]:
finaltest

,userID,movieID,tagWeight,genre,directorID,rating
0,0.0,0.010017,0.0,0.000000,0.814535,3.378571
1,0.0,0.010017,0.0,0.055556,0.814535,3.378571
2,0.0,0.010017,0.0,0.444444,0.814535,3.735714
3,0.0,0.023444,0.0,0.000000,0.631471,3.210714
4,0.0,0.023444,0.0,0.055556,0.631471,3.239286
...,...,...,...,...,...,...
189917,1.0,0.109556,0.0,0.222222,0.859787,3.257143
189918,1.0,0.109556,0.0,0.666667,0.859787,3.560714
189919,1.0,0.109556,0.0,0.777778,0.859787,3.407143
189920,1.0,0.550545,0.0,0.222222,0.567021,3.403571


In [45]:
# finaltest = finaltest.drop(columns=["tagWeight", "genre", "directorID", "actorID"], axis = 1)
finaltest = finaltest.drop(columns=["tagWeight", "genre", "directorID"], axis = 1)

In [46]:
finaltest

,userID,movieID,rating
0,0.0,0.010017,3.378571
1,0.0,0.010017,3.378571
2,0.0,0.010017,3.735714
3,0.0,0.023444,3.210714
4,0.0,0.023444,3.239286
...,...,...,...
189917,1.0,0.109556,3.257143
189918,1.0,0.109556,3.560714
189919,1.0,0.109556,3.407143
189920,1.0,0.550545,3.403571


In [47]:
# merge had created multiple rows of same userid and movieid due to multiple genres -> merged these rows with the final rating as the mean of all the ratings of that particular movieid and userid
finaltest = finaltest.groupby(['userID', 'movieID']).mean()

In [48]:
pred_list = list(finaltest["rating"])

In [49]:
pred_list

[3.497619047619048,
 3.2726190476190475,
 3.4017857142857144,
 3.458571428571428,
 3.5142857142857142,
 3.5785714285714287,
 3.462142857142857,
 3.4119047619047618,
 3.400892857142857,
 3.532142857142857,
 3.533333333333333,
 3.289285714285714,
 3.5200000000000005,
 3.5464285714285713,
 3.65,
 3.6464285714285714,
 3.3702380952380953,
 3.375,
 3.617857142857143,
 3.3517857142857146,
 3.532142857142857,
 3.5589285714285714,
 3.538095238095238,
 3.426190476190476,
 3.4482142857142857,
 3.3428571428571425,
 3.6375,
 3.6214285714285714,
 3.6589285714285715,
 3.4392857142857145,
 3.3499999999999996,
 3.5741071428571427,
 3.3714285714285714,
 3.4,
 3.2964285714285717,
 3.5232142857142854,
 3.430357142857143,
 3.3642857142857143,
 3.5440476190476193,
 3.617857142857143,
 3.4904761904761905,
 3.426190476190476,
 3.3580357142857142,
 3.4517857142857142,
 3.3380952380952382,
 3.4875000000000003,
 3.366071428571429,
 3.617857142857143,
 3.4169642857142857,
 3.461904761904762,
 3.5690476190476192,


In [50]:
finaltest["rating"].value_counts()

3.457143    381
3.432143    373
3.496429    356
3.471429    349
3.500000    348
           ... 
4.100000      1
4.096825      1
3.361310      1
3.759524      1
3.123810      1
Name: rating, Length: 3924, dtype: int64

In [51]:
pred_list[10]

3.533333333333333

In [52]:
pred_formatted_list = [ round(elem,1) for elem in pred_list ]

In [53]:
pred_formatted_list

[3.5,
 3.3,
 3.4,
 3.5,
 3.5,
 3.6,
 3.5,
 3.4,
 3.4,
 3.5,
 3.5,
 3.3,
 3.5,
 3.5,
 3.6,
 3.6,
 3.4,
 3.4,
 3.6,
 3.4,
 3.5,
 3.6,
 3.5,
 3.4,
 3.4,
 3.3,
 3.6,
 3.6,
 3.7,
 3.4,
 3.3,
 3.6,
 3.4,
 3.4,
 3.3,
 3.5,
 3.4,
 3.4,
 3.5,
 3.6,
 3.5,
 3.4,
 3.4,
 3.5,
 3.3,
 3.5,
 3.4,
 3.6,
 3.4,
 3.5,
 3.6,
 3.4,
 3.3,
 3.5,
 3.5,
 3.4,
 3.5,
 3.4,
 3.6,
 3.5,
 3.4,
 3.5,
 3.4,
 3.7,
 3.4,
 3.5,
 3.6,
 3.5,
 3.6,
 3.6,
 3.6,
 3.4,
 3.5,
 3.5,
 3.4,
 3.4,
 3.6,
 3.5,
 3.4,
 3.3,
 3.5,
 3.7,
 3.7,
 3.4,
 3.6,
 3.5,
 3.3,
 3.5,
 3.5,
 3.4,
 3.5,
 3.7,
 3.3,
 3.4,
 3.5,
 3.5,
 3.4,
 3.4,
 3.5,
 3.4,
 3.4,
 3.6,
 3.6,
 3.5,
 3.5,
 3.4,
 3.4,
 3.5,
 3.5,
 3.6,
 3.7,
 3.5,
 3.3,
 3.4,
 3.5,
 3.3,
 3.4,
 3.5,
 3.4,
 3.4,
 3.5,
 3.4,
 3.3,
 3.4,
 3.3,
 3.6,
 3.5,
 3.4,
 3.5,
 3.5,
 3.5,
 3.3,
 3.4,
 3.5,
 3.5,
 3.4,
 3.4,
 3.6,
 3.4,
 3.4,
 3.8,
 3.5,
 3.3,
 3.4,
 3.6,
 3.5,
 3.4,
 3.5,
 3.4,
 3.5,
 3.3,
 3.3,
 3.5,
 3.3,
 3.4,
 3.5,
 3.4,
 3.4,
 3.4,
 3.4,
 3.5,
 3.6,
 3.4,
 3.5,
 3.4,
 3.4,
 3.5

In [54]:
len(pred_formatted_list)

71299

In [55]:
pred_df = pd.DataFrame(pred_formatted_list,columns=['Column_Name'])
pred_df

,Column_Name
0,3.5
1,3.3
2,3.4
3,3.5
4,3.5
...,...
71294,3.5
71295,3.5
71296,3.5
71297,3.4


In [56]:
pred_df['Column_Name'].value_counts(sort=False)

3.5    21940
4.3       18
3.3    10046
3.0       94
3.1      521
3.6    10238
4.0      181
4.2       91
3.4    21360
4.1      127
3.9      336
3.7     2639
3.2     3045
3.8      644
2.9       19
Name: Column_Name, dtype: int64

In [57]:
with open('pred_values_actor.txt', 'w') as filehandle:
    for listitem in pred_formatted_list:
        filehandle.write('%s\n' % listitem)